# search and create db

In [ ]:
import os
import sys
import time

import pandas as pd
from papersdb import *

# https://chatgpt.com/c/a47b4d3e-bafc-419d-b067-eea1cdaa9ceb

search_phrases = [
    "Scalability challenges in integrated photonic switch networks",
    "Low-power design principles for photonic switch circuits",
    "High-bandwidth photonic switch network design strategies",
    "Latency optimization in photonic integrated circuits",
    "Modular design approaches for photonic switch networks",
    "Crossbar switch topologies in integrated photonic circuits",
    "Benes network architectures for scalable photonic switching",
    "Ring resonator-based photonic switch designs and applications",
    "Multistage network designs for integrated photonic circuits",
    "Single-mode vs multi-mode photonic switch network designs",
    "Mode-division multiplexing (MDM) in integrated photonics",
    "Spatial and wavelength division techniques in photonic switches",
    "Size and scaling considerations for photonic switch networks",
    "Miniaturization of photonic integrated circuits for large networks",
    "Technical specifications and trade-offs in photonic switches",
    "Insertion loss reduction in integrated photonic switch designs",
    "Crosstalk mitigation techniques in photonic circuits",
    "Switching speed optimization in photonic integrated networks",
    "Extinction ratio enhancement in photonic switch designs",
    "Waveguide design and optimization for photonic switch networks",
    "Material choices and trade-offs in photonic switch fabrication",
    "Silicon photonics-based integrated switch network designs",
    "MEMS-based photonic switches: design and performance",
    "Plasmonic components in integrated photonic switch networks",
    "Thermal tuning mechanisms in ring resonator switches",
    "Performance vs complexity in photonic switch network design",
    "AI/ML-based optimization for photonic circuit design",
    "Hybrid photonic-electronic switch network architectures",
    "Optical simulation tools for integrated photonic circuits",
    "Prototyping and testing in photonic switch network development",
    "Scalable photonic switch networks for data centers",
    "Design trade-offs in quantum computing with photonic switches",
    "Case studies of photonic switch networks in high-performance computing",
    "Future directions in integrated photonic switch network technologies",
    "Exploration of insertion loss and crosstalk in photonic switches",
    "Review of waveguide-based photonic switch technologies",
    "Nonlinear optical effects in photonic switch network design",
]

# https://claude.ai/chat/fee274fb-fea4-4ec9-9095-42a656f2f58e
wdm_search_phrases = [
    "Integrated photonic wavelength division multiplexer design principles",
    "WDM circuit design topologies for integrated photonics",
    "Multi-channel WDM architectures in silicon photonics",
    "Scaling challenges in integrated photonic WDM devices",
    "Technical specifications of on-chip wavelength division multiplexers",
    "Building blocks for integrated photonic WDM circuits",
    "Arrayed waveguide grating (AWG) design for WDM applications",
    "Microring resonator-based WDM multiplexers",
    "Echelle grating multiplexers in integrated photonics",
    "Mach-Zehnder interferometer WDM designs",
    "Adiabatic couplers for wavelength division multiplexing",
    "Fabrication techniques for integrated photonic WDM devices",
    "Optimization methods for WDM circuit design",
    "Insertion loss minimization in integrated WDM devices",
    "Crosstalk reduction techniques in photonic WDM multiplexers",
    "Thermal stability of integrated photonic WDM circuits",
    "Broadband WDM multiplexer designs for telecom applications",
    "Compact WDM devices for data center interconnects",
    "Polarization-independent WDM multiplexers",
    "Novel materials for integrated photonic WDM applications",
]


# 'integrated silicon photonics switch networks'


df = recursive_paper_search(wdm_search_phrases, depth=0, papers_N=800)
print(df.info())
df.to_parquet("db/papers_wdm.parquet", index=False)
# all_papers_df.to_csv('papers.csv', index=False)

# download open access pdfs

In [ ]:
from papersdb import *

all_papers_df = pd.read_parquet("db/papers_wdm.parquet")

print("------ Downloading PDFs")
i = j = 0
for index, row in all_papers_df.iterrows():
    pdf_url = row["openAccessPdf"]
    if pdf_url:  # Check if the PDF URL exists
        if os.path.exists(f"db/pdfs/{row['paperId']}.pdf"):
            i += 1
            print("file exists!")
        else:
            r = download_pdf_requests(pdf_url, "db/pdfs_wdm", row["paperId"])
            if r is not None:
                j += 1
            time.sleep(2)  # Adjust sleep time if necessary

print("====== DONE! ======")
print("pdfs already exsited: ", i)
print("pdfs downloaded", j)

# Checking data

In [ ]:
import os
import time

import pandas as pd
from papersdb import *

df = pd.read_parquet("papers.parquet")
print(df.shape)
# df.info()

# df_sorted = df.sort_values(by='influentialCitationCount', ascending=False)
# for index, row in df_sorted.head(100).iterrows():
#     print(row['influentialCitationCount'], row['url'])

arx = 0
doi = 0
PubMed = 0
for index, row in df.iterrows():
    externalIds = row["externalIds"]
    if externalIds["ArXiv"]:
        arx += 1
    if externalIds["DOI"]:
        doi += 1
    if externalIds["PubMed"]:
        PubMed += 1

print("arx", arx)
print("doi", doi)
print("PubMed", PubMed)

# {'ArXiv': None, 'CorpusId': 125852517, 'DBLP': None, 'DOI': '10.1117/12.2255794', 'MAG': '2582829612', 'PubMed': None, 'PubMedCentral': None}

# download pdfs by doi link
based on the cell above, about 90% of these paper (about switch networks) havd doi links. let's use that to download pdfs.

Downloading PDFs proved to be challenging. Many sites are easy for a bot like below, but many others like IEEE and optica block a like this bot. 

plan:
- doi link
- arrive at the landing page
- save the landing url in the db
- download the html page as {html, mhtml, pdf}?
- use llm to find the pdf download link
- dowanlod pdf

Mostly works! 
Problem: Saving HTML of landing pages is not always reliable because dynamic content like JS changes the links and not valid.
possible solutions:
- save landing page as mhtml 
    - none of these python tools can get a mhtml reliably: playwright, pyppeteer, selenium, requests_html. 
    - since they are mostly in JS, i tried pyppeteer with node. mhtml works pretty well. but it's not loading optica pages!
- save landing pages as pdf
    - more straightforward
    - solves the main issue
    - is not as nice as the idea of mhtml!

i think for optica and maybe many more, if you know the landing url, you know the pdf url.


In [ ]:
# landing_html_url.py

used the script above to save landing pages in html and pdf and add landing url to the parquet df.

Now checking the landing urls:

In [ ]:
from urllib.parse import urlparse

import pandas as pd

df = pd.read_parquet("db/papers_wdm.parquet")
print("shape of db:", df.shape)
print()

df["domain"] = df["landing_url"].apply(lambda x: urlparse(x).netloc)
domain_counts = df["domain"].value_counts()

print("domains:")
print(domain_counts[:20])

# df.info()

# count how many have DOI, landing url, pdf

In [ ]:
import os

import pandas as pd

df = pd.read_parquet("db/papers_switch.parquet")

i = j = k = l = m = n = o = p = 0
for index, row in df.iterrows():
    doiurl = row["externalIds"]["DOI"]
    doi_exist = doiurl is not None
    file_exist = os.path.exists(f"db/pdfs/{row['paperId']}.pdf")
    if not file_exist:
        i += 1
    if doi_exist:
        j += 1
    if pd.isna(row["landing_url"]):
        k += 1
    if doi_exist & (not file_exist):
        l += 1
    if doi_exist & pd.isna(row["landing_url"]):
        m += 1
    if doi_exist & (not file_exist) & pd.isna(row["landing_url"]):
        n += 1
    if row["landing_url"]:
        if "optica.org" in row["landing_url"]:
            o += 1
            if not file_exist:
                p += 1
                # print(row['landing_url'], row['paperId'])
                # print()

print("df shape", df.shape)
print("missing pdf files", i)
print("doi links", j)
print("missing landing urls", k)
print("missing pdf but doi exist", l)
print("missing landing url but doi exist", m)
print("missing landing url, missing file, but doi exist", n)
print("optica landing urls", o)
print("optica landing urls but pdf missing", p)
# print('pdf files', i)

# download from IEEE and optica

In [ ]:
# see ieee_pdf.py

# optica_pdf.py doesn't work!

# download papers that have arxiv id

In [ ]:
import os
import time

import pandas as pd
from papersdb import *

df = pd.read_parquet("db/papers_wdm.parquet")

print("------ Downloading arxiv PDFs")
for index, row in df.iterrows():
    ids = row["externalIds"]
    arxID = ids["ArXiv"]
    if arxID is not None:
        if not os.path.exists(f"db/pdfs/{row['paperId']}.pdf"):
            r = download_pdf_requests(
                f"https://arxiv.org/pdf/{arxID}", "db/pdfs", row["paperId"]
            )

            df.at[index, "landing_url"] = f"https://arxiv.org/pdf/{arxID}"
            df.to_parquet("db/postURL_papers.parquet", index=False)

            time.sleep(10)

print("====== DONE! ======")

# fetch more landing pages

In [ ]:
from pypdf import PdfReader, PdfWriter

# Open the original PDF file
with open("db/6879eb6ad9a2fc1a55a6a07ea2e7a72c8dd642a0.pdf", "rb") as input_pdf_file:
    pdf_reader = PdfReader(input_pdf_file)
    pdf_writer = PdfWriter()

    # Add the first two pages to the writer
    if len(pdf_reader.pages) >= 2:
        pdf_writer.add_page(pdf_reader.pages[0])
        pdf_writer.add_page(pdf_reader.pages[1])
    else:
        print("The PDF does not have two pages.")

    # Save the modified PDF to a new file
    with open("db/output.pdf", "wb") as output_pdf_file:
        pdf_writer.write(output_pdf_file)

print("The PDF has been processed and saved as 'output.pdf'.")

In [ ]:
import os
import time

import pandas as pd
from papersdb import *

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
from PhotonicsAI.Photon import llm_api

# pdf_url = 'https://www.spiedigitallibrary.org/conference-proceedings-of-spie/10131/1013104/The-role-of-integrated-photonics-in-datacenter-networks/10.1117/12.2255794.pdf'
# r = download_pdf_with_fallback(pdf_url, 'pdfs', '00000000000000')
# print(r)

sys_prompt = 'This is a webpage of a scientific article. Can you find a PDF download link? If yes, only return the link with no other information. If no, only return False."'
# html_content = await save_page_as_html('https://www.doi.org/10.1117/12.2255794', 'example.html')
# r = llm_api.call_llm(html_content, sys_prompt, llm_api_selection='gpt-4o-mini')
# print(r)

all_papers_df = pd.read_parquet("papers.parquet")

for index, row in all_papers_df.iterrows():
    externalIds = row["externalIds"]
    if externalIds["DOI"]:
        if os.path.exists(f'pdfs/{row['paperId']}.pdf'):
            # print('file exists!')
            pass
        else:
            url = "https://www.doi.org/" + externalIds["DOI"]
            print("____________")
            print("DOI:", url)
            html_content, landing_url = await save_page_as_html(
                url, f'doihtml/{row['paperId']}.html'
            )
            if html_content:
                print("landing url:", landing_url)
                llm_r = llm_api.call_llm(
                    html_content, sys_prompt, llm_api_selection="gpt-4o-mini"
                )
                print("pdf link by llm:", llm_r)
                if is_valid_url(llm_r):
                    r = download_pdf_requests(llm_r, "pdfs", row["paperId"])
                else:
                    print("invalid link?!")
                time.sleep(2)  # Adjust sleep time if necessary